In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from random import shuffle
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import cross_val_score
import pickle
import time
from hyperopt import fmin, tpe, hp, space_eval, rand, Trials, partial, STATUS_OK

C:\Anaconda2\envs\python36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [59]:
target='Disbursed'
IDcol = 'ID'
predictors = [x for x in traindata.columns if x not in [target, IDcol]]

In [54]:
trainfilePath = r"C:\Users\dell\Desktop\train_modified.csv"
testfilePath = r"C:\Users\dell\Desktop\test_modified.csv"
traindata = pd.read_csv(trainfilePath)
testdata = pd.read_csv(testfilePath)

In [55]:
traindata[:6]

,Disbursed,Existing_EMI,ID,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,...,Var2_2,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2
0,0.0,0.0,ID000002C20,300000.0,5.0,20000,1,0,37,1,...,0,0,0,0,1,1,0,1,0,0
1,0.0,0.0,ID000004E40,200000.0,2.0,35000,3,13,30,0,...,0,0,0,0,1,0,1,1,0,0
2,0.0,0.0,ID000007H20,600000.0,4.0,22500,1,0,34,1,...,0,0,0,0,0,0,1,0,0,1
3,0.0,0.0,ID000008I30,1000000.0,5.0,35000,3,10,28,1,...,0,0,0,0,0,0,1,0,0,1
4,0.0,25000.0,ID000009J40,500000.0,2.0,100000,3,17,31,1,...,0,0,0,0,0,0,1,0,0,1
5,0.0,15000.0,ID000010K00,300000.0,5.0,45000,3,17,33,0,...,0,0,0,0,0,0,1,0,0,1


In [57]:
testdata[:6]

,Existing_EMI,ID,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,Interest_Rate_Missing,...,Var2_2,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2
0,0.0,ID000026A10,100000.0,3.0,21500,3,3,28,0,0,...,0,0,0,0,0,0,1,1,0,0
1,0.0,ID000054C40,0.0,0.0,42000,5,8,35,0,0,...,1,0,0,0,0,0,1,0,1,0
2,0.0,ID000066O10,300000.0,2.0,10000,1,0,26,1,1,...,0,0,0,0,0,1,0,0,1,0
3,0.0,ID000110G00,0.0,0.0,14650,1,0,24,1,1,...,1,0,0,0,0,1,0,0,1,0
4,5000.0,ID000113J30,100000.0,1.0,23400,1,0,28,1,1,...,0,0,0,0,0,0,1,0,0,1
5,4500.0,ID000133D30,100000.0,0.0,15000,1,0,29,1,1,...,0,0,0,0,0,1,0,0,1,0


In [61]:
train_x=traindata[predictors]
train_y = traindata[target]
test_x=testdata[predictors]

In [45]:
text_x

,Existing_EMI,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,Interest_Rate_Missing,Loan_Amount_Submitted_Missing,...,Var2_2,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2
0,0.0,300000.0,5.0,20000,1,0,37,1,1,1,...,0,0,0,0,1,1,0,1,0,0
1,0.0,200000.0,2.0,35000,3,13,30,0,0,0,...,0,0,0,0,1,0,1,1,0,0
2,0.0,600000.0,4.0,22500,1,0,34,1,1,0,...,0,0,0,0,0,0,1,0,0,1
3,0.0,1000000.0,5.0,35000,3,10,28,1,1,0,...,0,0,0,0,0,0,1,0,0,1
4,25000.0,500000.0,2.0,100000,3,17,31,1,1,0,...,0,0,0,0,0,0,1,0,0,1
5,15000.0,300000.0,5.0,45000,3,17,33,0,0,0,...,0,0,0,0,0,0,1,0,0,1
6,0.0,6.0,5.0,70000,1,0,28,1,1,1,...,0,0,0,0,0,1,0,0,1,0
7,2597.0,200000.0,5.0,20000,3,3,40,1,1,0,...,0,0,0,0,0,0,1,0,0,1
8,0.0,0.0,0.0,75000,5,13,43,0,0,0,...,1,0,0,0,0,0,1,1,0,0
9,0.0,300000.0,3.0,30000,1,0,26,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [17]:
train_x

,Existing_EMI,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,Interest_Rate_Missing,Loan_Amount_Submitted_Missing,...,Var2_2,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2
0,0.0,300000.0,5.0,20000,1,0,37,1,1,1,...,0,0,0,0,1,1,0,1,0,0
1,0.0,200000.0,2.0,35000,3,13,30,0,0,0,...,0,0,0,0,1,0,1,1,0,0
2,0.0,600000.0,4.0,22500,1,0,34,1,1,0,...,0,0,0,0,0,0,1,0,0,1
3,0.0,1000000.0,5.0,35000,3,10,28,1,1,0,...,0,0,0,0,0,0,1,0,0,1
4,25000.0,500000.0,2.0,100000,3,17,31,1,1,0,...,0,0,0,0,0,0,1,0,0,1
5,15000.0,300000.0,5.0,45000,3,17,33,0,0,0,...,0,0,0,0,0,0,1,0,0,1
6,0.0,6.0,5.0,70000,1,0,28,1,1,1,...,0,0,0,0,0,1,0,0,1,0
7,2597.0,200000.0,5.0,20000,3,3,40,1,1,0,...,0,0,0,0,0,0,1,0,0,1
8,0.0,0.0,0.0,75000,5,13,43,0,0,0,...,1,0,0,0,0,0,1,1,0,0
9,0.0,300000.0,3.0,30000,1,0,26,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [20]:
train_y

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
86990    0.0
86991    0.0
86992    0.0
86993    0.0
86994    0.0
86995    0.0
86996    0.0
86997    0.0
86998    0.0
86999    0.0
87000    0.0
87001    0.0
87002    0.0
87003    0.0
87004    0.0
87005    0.0
87006    0.0
87007    0.0
87008    0.0
87009    0.0
87010    0.0
87011    0.0
87012    0.0
87013    0.0
87014    0.0
87015    0.0
87016    0.0
87017    0.0
87018    0.0
87019    0.0
Name: Disbursed, Length: 87020, dtype: float64

In [34]:
def GBM(argsDict):
    max_depth = argsDict["max_depth"]
    n_estimators = argsDict['n_estimators'] 
    learning_rate = argsDict["learning_rate"]
    subsample = argsDict["subsample"] 
    min_child_weight = argsDict["min_child_weight"]
    print("max_depth:" + str(max_depth))
    print("n_estimator:" + str(n_estimators))
    print("learning_rate:" + str(learning_rate))
    print("subsample:" + str(subsample))
    print("min_child_weight:" + str(min_child_weight))
    global attr_train, label_train

    gbm = xgb.XGBClassifier(nthread=4,  # 进程数
                            max_depth=max_depth,  # 最大深度
                            n_estimators=n_estimators,  # 树的数量
                            learning_rate=learning_rate,  # 学习率
                            subsample=subsample,  # 采样数
                            min_child_weight=min_child_weight,  # 孩子数
                            max_delta_step=10,  # 10步不降则停止
                            objective="binary:logistic")

    metric = cross_val_score(gbm, train_x, train_y,
                             cv=5, scoring="roc_auc").mean()
    print(metric)
    return -metric

In [35]:
space = {"max_depth": hp.randint("max_depth", 15),
         # [0,1,2,3,4,5] -> [50,]
         "n_estimators": hp.randint("n_estimators", 10),
         # [0,1,2,3,4,5] -> 0.05,0.06
         "learning_rate": hp.randint("learning_rate", 6),
         # [0,1,2,3] -> [0.7,0.8,0.9,1.0]
         "subsample": hp.randint("subsample", 4),
         "min_child_weight": hp.randint("min_child_weight", 5),
         }

In [36]:
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(GBM, space, algo=algo, max_evals=4)

max_depth:8
n_estimator:50
learning_rate:0.11
subsample:0.7999999999999999
min_child_weight:4
0.836528962717
max_depth:8
n_estimator:50
learning_rate:0.11
subsample:0.7999999999999999
min_child_weight:4
0.836528962717
max_depth:19
n_estimator:75
learning_rate:0.09
subsample:0.7
min_child_weight:3
0.827491729024
max_depth:15
n_estimator:65
learning_rate:0.11
subsample:0.7999999999999999
min_child_weight:2
0.822973265664


In [38]:
print(best)
print(GBM(best))

{'learning_rate': 3, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 0, 'subsample': 1}
max_depth:8
n_estimator:50
learning_rate:0.11
subsample:0.7999999999999999
min_child_weight:4
0.836528962717
-0.836528962717


In [29]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

C:\Anaconda2\envs\python36\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [75]:
def modelfit(alg, dtrain, dtest, predictors, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):

    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(
            dtrain[predictors].values, label=dtrain[target].values)
        xgtest = xgb.DMatrix(dtest[predictors].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          metrics='auc', early_stopping_rounds=early_stopping_rounds, show_stdv=False)
        alg.set_params(n_estimators=cvresult.shape[0])

    # Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'], eval_metric='auc')

    # Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:, 1]

    # Print model report:
    print("\nModel Report")
    print(("Accuracy : %.4f") % metrics.accuracy_score(
        dtrain['Disbursed'].values, dtrain_predictions))
    print(("AUC Score (Train): %f") % metrics.roc_auc_score(
        dtrain['Disbursed'], dtrain_predprob))

#     Predict on testing data:
    dtest['predprob'] = alg.predict_proba(dtest[predictors])[:, 1]
    results = test_results.merge(dtest[['ID', 'predprob']], on='ID')
    print(('AUC Score (Test): %f') % metrics.roc_auc_score(results['Disbursed'], results['predprob']))

    feat_imp = pd.Series(alg.booster().get_fscore()
                         ).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')


In [76]:
alg = XGBClassifier(
    learning_rate=0.01,
    n_estimators=5000,
    max_depth=4,
    min_child_weight=6,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.005,
    objective='binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27)

In [77]:
modelfit(xgb1, traindata, testdata, predictors)


Model Report
Accuracy : 0.9854
AUC Score (Train): 0.884030


NameError: name 'test_results' is not defined

In [64]:
xgb_param = alg.get_xgb_params()

In [65]:
xgb_param

{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.01,
 'max_delta_step': 0,
 'max_depth': 4,
 'min_child_weight': 6,
 'missing': None,
 'n_estimators': 5000,
 'nthread': 4,
 'objective': 'binary:logistic',
 'reg_alpha': 0.005,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 27,
 'silent': 1,
 'subsample': 0.8}

In [67]:
xgtrain = xgb.DMatrix(
            traindata[predictors].values, label=traindata[target].values)